In [4]:
import psycopg2

# Step 1: Connect to the bank_reviews database
conn = psycopg2.connect(
    dbname="bank_reviews",  # <-- make sure to connect to your database
    user="postgres",
    password="7121",        # your postgres password
    host="localhost",
    port="5432"
)

cur = conn.cursor()

# Step 2: Create Banks table
cur.execute("""
CREATE TABLE IF NOT EXISTS banks (
    bank_id SERIAL PRIMARY KEY,
    bank_name VARCHAR(255) NOT NULL,
    app_name VARCHAR(255)
);
""")

# Step 3: Create Reviews table
cur.execute("""
CREATE TABLE IF NOT EXISTS reviews (
    review_id SERIAL PRIMARY KEY,
    bank_id INT NOT NULL REFERENCES banks(bank_id) ON DELETE CASCADE,
    review_text TEXT,
    rating NUMERIC(2,1),
    review_date DATE,
    sentiment_label VARCHAR(20),
    sentiment_score NUMERIC(3,2),
    source VARCHAR(50)
);
""")

# Step 4: Commit changes and close connection
conn.commit()
cur.close()
conn.close()

print("Tables 'banks' and 'reviews' created successfully in 'bank_reviews' database!")


Tables 'banks' and 'reviews' created successfully in 'bank_reviews' database!
